### CLUSTERING TORONTO NEIGHBORHOODS ON THE BASIS OF SIMILAR VENUES


In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print('libararies imported')

libararies imported


### 1. getting data and and tranforming it to dataframe 

#### using https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M to get list of neighborhoods in toronto


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_text = requests.get(url).text
print(url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [3]:
soup = BeautifulSoup(html_text, 'lxml')
table = soup.find('table')
trs = table.find_all('tr')

rows=[]
for tr in trs:
    row=tr.find_all('td')
    if row:
        rows.append(row)

lst=[]
for row in rows:
    postal_code=row[0].text.rstrip()
    borough=row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    
    if borough!='Not assigned':
        if neighborhood=='Not assigned':
            neighborhood = borough
        lst.append([postal_code, borough, neighborhood])
lst

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

In [4]:
df = pd.DataFrame(lst ,columns=['PostalCode','Borough','Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
df.shape

(103, 3)

In [6]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [7]:
#!pip install pgeocode
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['PostalCode'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
        
    

Postal Code M3A has been retrieved. 102 codes left
Postal Code M4A has been retrieved. 101 codes left
Postal Code M5A has been retrieved. 100 codes left
Postal Code M6A has been retrieved. 99 codes left
Postal Code M7A has been retrieved. 98 codes left
Postal Code M9A has been retrieved. 97 codes left
Postal Code M1B has been retrieved. 96 codes left
Postal Code M3B has been retrieved. 95 codes left
Postal Code M4B has been retrieved. 94 codes left
Postal Code M5B has been retrieved. 93 codes left
Postal Code M6B has been retrieved. 92 codes left
Postal Code M9B has been retrieved. 91 codes left
Postal Code M1C has been retrieved. 90 codes left
Postal Code M3C has been retrieved. 89 codes left
Postal Code M4C has been retrieved. 88 codes left
Postal Code M5C has been retrieved. 87 codes left
Postal Code M6C has been retrieved. 86 codes left
Postal Code M9C has been retrieved. 85 codes left
Postal Code M1E has been retrieved. 84 codes left
Postal Code M4E has been retrieved. 83 codes le

In [9]:
df['Latitude']=latitudes
df['Longitude']=longitudes
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939
